In [ ]:
import numpy as np
import torch
import os
from torch import nn
import torch.nn.functional as F
from glob import glob

In [ ]:
model_path = '../ncnn_aka/ncnn_aka/output'

# 生成网络结构

In [ ]:
# get layer信息
layers = {}
bottoms = {}
tops = {}
param = {}
index = {}
layers_file = glob(model_path+'/*_*.txt')
for layer in layers_file:
    layer_type = layer.split('_')[-1][:-4]
    layer_index = int(os.path.basename(layer).split('_')[0])
    with open(layer, "r") as f:
        layer_name = f.readline().strip('\n')
        layer_bottoms = f.readline().split(' ')[:-1]
        layer_tops = f.readline().split(' ')[:-1]
        layer_param = f.readline().strip('\n')
    bottoms[layer_name] = layer_bottoms
    tops[layer_name] = layer_tops
    layers[layer_name] = layer_type
    param[layer_name] = layer_param
    index[layer_index] = layer_name

In [ ]:
# get network信息
with open(model_path+'/network.txt', "r") as f:
    all = f.readlines()
    input_names = all[0].split(' ')[:-1]
    output_names = [x.split(' ')[:-1] for x in all[1:]]

In [ ]:
pre = "        "
end = "\n"
code = "\
from torch import nn\n\
from utils import *\n\
class fuck(nn.Module):\n\
    def __init__(self):\n\
        super(fuck, self).__init__()\n\
\n\
{}\n\
\n\
    def forward(self, {}):\n\
{}\n\
\n\
{}\n\
        return {}"

In [ ]:
fcode1 = ','.join(input_names)

fcode2 = pre+"blob_mat = {}"+end
for i in input_names:
    fcode2 += pre+"blob_mat[\'"+i+"\'] = "+i

fcode3 = ""
for output1 in output_names:
    output_blob = output1[0]
    calculate_pipeline = output1[1:]
    for l in calculate_pipeline:
        f = layers[l]
        i = ','.join(['blob_mat[\''+x+'\']' for x in bottoms[l]])
        o = ','.join(['blob_mat[\''+x+'\']' for x in tops[l]])
        fcode3 += pre+"{} = {}({})".format(o,"self.layers_"+l.replace('/','_'),i)+end
    fcode3 += end
        
fcode4 = ','.join(['blob_mat[\''+x[0]+'\']' for x in output_names])

In [ ]:
icode2 = ""
for layer_name,layer_type in layers.items():
    layer_param = param[layer_name]
    icode2 += pre+"self.layers_{} = {}(\'{}\')".format(layer_name.replace('/','_'),layer_type,layer_param)+end

In [ ]:
f = open("fuck.py", "w")
print(code.format(icode2,fcode1,fcode2,fcode3,fcode4), file = f)
f.close()

# 加载权重

In [ ]:
from fuck import *
model = fuck()

In [ ]:
# 加载权重参数
bin_files = glob(model_path+'/*_*_weight.bin')

In [ ]:
for bin_file in bin_files:
    bin_file_name = os.path.basename(bin_file)
    bin_index = int(bin_file_name.split('_')[0])
    bin_layer_name = 'layers_'+index[bin_index].replace('/','_')
    
    if layers[index[bin_index]] == 'Convolution':
        conv_layer = getattr(model,bin_layer_name)[0]
        # 加载weight
        w = np.fromfile(bin_file,dtype=np.float32)
        w = w.reshape(conv_layer.out_channels,conv_layer.in_channels,*conv_layer.kernel_size)
        w = nn.Parameter(data=torch.FloatTensor(w))
        conv_layer.weight = w
        # 加载bias
        if conv_layer.bias is not None:
            bias_bin_file_name = bin_file.replace('weight','bias')
            b = np.fromfile(bias_bin_file_name,dtype=np.float32)
            b = nn.Parameter(data=torch.FloatTensor(b))
            conv_layer.bias = b

In [ ]:
torch.save(model.state_dict(),'fuck.pth')

In [ ]:
torch.onnx.export(model,torch.rand(1,3,227,227),'fuck.onnx')